In [18]:
import ast
import copy
import re

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from aocd import get_data, submit

%load_ext line_profiler

DAY = 17
YEAR = 2022

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [19]:
# use test data
raw_test = """>>><<><>><<<>><>>><<<>>><<<><<<>><>><<>>"""

# use real data
raw = get_data(day=DAY, year=YEAR)

print(raw_test)

>>><<><>><<<>><>>><<<>>><<<><<<>><>><<>>


In [20]:
def parse_data(data):
    data = list(data)
    return np.array(data)


dummy = parse_data(raw_test)
real = parse_data(raw)

dummy

array(['>', '>', '>', '<', '<', '>', '<', '>', '>', '<', '<', '<', '>',
       '>', '<', '>', '>', '>', '<', '<', '<', '>', '>', '>', '<', '<',
       '<', '>', '<', '<', '<', '>', '>', '<', '>', '>', '<', '<', '>',
       '>'], dtype='<U1')

# Part 1

In [21]:
def intersects(cave, rock):
    global width
    if not cave.isdisjoint(rock):
        return True

    for x, _ in rock:
        if x < 0 or x >= 7:
            return True

    return False


def push(cave, rock, jet):
    if jet == "<":
        new_rock = {(x - 1, y) for x, y in rock}
        return new_rock if not intersects(cave, new_rock) else rock

    new_rock = {(x + 1, y) for x, y in rock}
    return new_rock if not intersects(cave, new_rock) else rock


def init_rock(rock, start_y):
    return {(x, y + start_y) for x, y in rock}


def drop(cave, rock):
    new_rock = {(x, y - 1) for x, y in rock}
    if not intersects(cave, new_rock):
        return new_rock, False

    return rock, True


def drop_rock(cave, rock, jets, jet_start_idx, highest_point):
    start_y = highest_point + 4
    jet_idx = jet_start_idx

    rock = init_rock(rock, start_y)

    while True:
        rock = push(cave, rock, jets[jet_idx % len(jets)])
        rock, has_stopped = drop(cave, rock)
        jet_idx += 1

        if has_stopped:
            highest_point = max({y for _, y in rock} | {highest_point})
            return cave | rock, rock, jet_idx, highest_point

In [22]:
def run_simulation(jets, rocks_to_drop):
    n_rocks = 0
    jet_idx = 0
    max_y = 0
    width = 7
    lj = len(jets)

    bottom = [(x, 0) for x in range(0, width)]
    cave = set(bottom.copy())
    rocks = [
        [(0, 0), (1, 0), (2, 0), (3, 0)],
        [(1, 0), (0, 1), (1, 1), (2, 1), (1, 2)],
        [(0, 0), (1, 0), (2, 0), (2, 1), (2, 2)],
        [(0, 0), (0, 1), (0, 2), (0, 3)],
        [(0, 0), (1, 0), (0, 1), (1, 1)],
    ]
    rocks = [{(x + 2, y) for x, y in r} for r in rocks]

    while True:
        rock = rocks[n_rocks % 5]
        cave, rock, jet_idx, max_y = drop_rock(cave, rock, jets, jet_idx, max_y)
        n_rocks += 1

        if n_rocks == rocks_to_drop:
            return max_y


jets = real.copy()

result = run_simulation(jets, 2022)
result

3090

# Part 2

In [23]:
def run_simulation_v2(jets, goal, rocks_to_drop=None):
    n_rocks = 0
    jet_idx = 0
    max_y = 0
    width = 7

    bottom = [(x, 0) for x in range(0, width)]
    cave = set(bottom.copy())
    rocks = [
        [(0, 0), (1, 0), (2, 0), (3, 0)],
        [(1, 0), (0, 1), (1, 1), (2, 1), (1, 2)],
        [(0, 0), (1, 0), (2, 0), (2, 1), (2, 2)],
        [(0, 0), (0, 1), (0, 2), (0, 3)],
        [(0, 0), (1, 0), (0, 1), (1, 1)],
    ]
    rocks = [{(x + 2, y) for x, y in r} for r in rocks]

    patterns = {}
    values = []

    while True:
        rock = rocks[n_rocks % 5]
        cave, rock, jet_idx, max_y = drop_rock(cave, rock, jets, jet_idx, max_y)
        n_rocks += 1

        # search for periods
        for jdx in range(len(jets)):
            for rdx in range(len(rocks)):
                if jet_idx % len(jets) == jdx and n_rocks % len(rocks) == rdx:
                    if (jdx, rdx) not in patterns:
                        patterns[(jdx, rdx)] = (n_rocks, max_y)
                    else:
                        old_rocks, old_height = patterns[(jdx, rdx)]
                        rdiff, hdiff = n_rocks - old_rocks, max_y - old_height
                        result = int(
                            ((goal - old_rocks) // rdiff) * hdiff
                            + run_simulation(jets, old_rocks + (goal - old_rocks) % rdiff)
                        )
                        values.append(result)

                        if len(values) > 10 and len(np.unique(values[-10:])) == 1:
                            return result


jets = real.copy()

result = run_simulation_v2(jets, 1e12)
result

1530057803453

 ### Improved

In [24]:
def run_simulation_v2(jets, goal):
    n_rocks = 0
    jet_idx = 0
    max_y = 0
    width = 7

    bottom = [(x, 0) for x in range(0, width)]
    cave = set(bottom.copy())
    rocks = [
        [(0, 0), (1, 0), (2, 0), (3, 0)],
        [(1, 0), (0, 1), (1, 1), (2, 1), (1, 2)],
        [(0, 0), (1, 0), (2, 0), (2, 1), (2, 2)],
        [(0, 0), (0, 1), (0, 2), (0, 3)],
        [(0, 0), (1, 0), (0, 1), (1, 1)],
    ]
    rocks = [{(x + 2, y) for x, y in r} for r in rocks]

    scan = []
    state_hash_dict = {}
    while True:
        rock = rocks[n_rocks % 5]
        cave, rock, jet_idx, max_y = drop_rock(cave, rock, jets, jet_idx, max_y)
        n_rocks += 1

        if len(scan) < 100:
            scan += [rock]
        else:
            scan = scan[1:] + [rock]
            my = max((y for r in scan for _, y in r))
            pattern = tuple(sorted([(x, y - my) for r in scan for (x, y) in r], key=lambda x: (x[0], x[1])))
            period = (jet_idx % len(jets), n_rocks % len(rocks))
            state_hash = hash(pattern + period)

            if state_hash in state_hash_dict:
                old_n_rocks, old_max_y = state_hash_dict[state_hash]
                r_diff, y_diff = n_rocks - old_n_rocks, max_y - old_max_y
                result = int(
                    ((goal - old_n_rocks) // r_diff) * y_diff
                    + run_simulation(jets, old_n_rocks + (goal - old_n_rocks) % r_diff)
                )
                return result

            state_hash_dict[state_hash] = (n_rocks, max_y)


jets = real.copy()

result = run_simulation_v2(jets, 1e12)
result

1530057803453

In [25]:
# submit(result, part="b", day=DAY, year=YEAR)